# Using T5 Small 80M for Airflow Docs Q&A

## Dependencies

In [1]:
%pip install accelerate transformers[torch] torch sentencepiece --user

Note: you may need to restart the kernel to use updated packages.


## Model Setup

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

<pad> Wie ich er bitten?</s>


/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
import os
from pathlib import Path

models_path = Path(f"{os.getcwd()}/models/t5-small")
if not models_path.exists():
 models_path.mkdir(parents=True, exist_ok=True)

# Storing tokenizer locally
tokenizer.save_pretrained(str(models_path))
print("Tokenizer saved successfully!")
# Storing model locally
model.save_pretrained(str(models_path))
print("Model saved successfully!")

Tokenizer saved successfully!
Model saved successfully!


## Loading stored model

In [18]:
import os
from pathlib import Path
from transformers import T5Tokenizer, T5ForConditionalGeneration

models_path = Path(f"{os.getcwd()}/models/t5-small")
tokenizer = T5Tokenizer.from_pretrained(str(models_path))
model = T5ForConditionalGeneration.from_pretrained(str(models_path))

## Documents URL Scraping

In [19]:
%pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [20]:
from pathlib import Path
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests

root_url = "https://airflow.apache.org/docs/apache-airflow/stable/index.html"
root_response = requests.get(root_url)
root_html = root_response.content.decode("utf-8")
soup = BeautifulSoup(root_html, 'html.parser')

root_url_parts = urlparse(root_url)
root_links = soup.find_all("a", attrs={"class": "reference internal"})

result = set()
for root_link in root_links:
    path = root_url_parts.path + root_link.get("href")
    path = str(Path(path).resolve())
    path = urlparse(path).path  # remove the hashtag
    url = f"{root_url_parts.scheme}://{root_url_parts.netloc}{path}"
    if not url.endswith("/"):
        url = url + "/"
    result.add(url)
print(list(result))

['https://airflow.apache.org/docs/apache-airflow/stable/index.htmlhowto/index.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmltemplates-ref.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmldeprecated-rest-api-ref.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmltutorial/index.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlconfigurations-ref.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlfaq.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmllicense.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlintegration.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlstable-rest-api-ref.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlui.html/', 'https://airflow.apache.org/docs/apache-airflow/stable/index.htmlrele

## Embeddings Database

In [21]:
%pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.1 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 22.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
from pathlib import Path
from langchain import HuggingFacePipeline

models_path = Path(f"{os.getcwd()}/models/t5-small")
llm = HuggingFacePipeline.from_model_id(
    model_id=str(models_path),
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64},
)

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, RwkvConfig, Speech2Text2Config, TransfoXLConfig, TrOCRConfig, XGLMConfig, XLMConfig, XLMProphetNetConfig, XLMRobertaConfig, XLMRobertaXLConfig, XLNetC

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


CHROMA_DB_DIRECTORY = "chroma_db/ask_django_docs"

loader = WebBaseLoader(urls)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splitted_documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(
    splitted_documents,
    embeddings,
    collection_name="ask_django_docs",
    persist_directory=CHROMA_DB_DIRECTORY,
)
db.persist()